In [53]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing
%matplotlib inline



In [54]:
macro_cols = ["balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract",
              "micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate",
              "income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build"]


In [55]:
df_train = pd.read_csv("/Users/yangyangwang/kaggle/Sberbank-Russian-Housing-Market/train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("/Users/yangyangwang/kaggle/Sberbank-Russian-Housing-Market/test.csv", parse_dates=['timestamp'])
#df_fix=pd.read_excel("/Users/yangyangwang/kaggle/Sberbank-Russian-Housing-Market/BAD_ADDRESS_FIX.xlsx").set_index('id')
df_macro = pd.read_csv("/Users/yangyangwang/kaggle/Sberbank-Russian-Housing-Market/macro.csv", parse_dates=['timestamp'],usecols=['timestamp'] + macro_cols)
print df_train.shape
print df_test.shape
#print df_fix.shape
print df_macro.shape


(30471, 292)
(7662, 291)
(2484, 14)


In [56]:
ylog_train_all = np.log1p(df_train['price_doc'].values)
id_test = df_test['id']
print df_train['price_doc']
print ylog_train_all

0         5850000
1         6000000
2         5700000
3        13100000
4        16331452
5         9100000
6         5500000
7         2000000
8         5300000
9         2000000
10        4650000
11        4800000
12        5100000
13        5200000
14        5000000
15        1850000
16        6300000
17        5900000
18        7900000
19        5200000
20        8200000
21        5200000
22        6250000
23        5750000
24        6000000
25        1050000
26        5000000
27        4700000
28        8254400
29        5900000
           ...   
30441    25039300
30442     4350000
30443     7567425
30444    11438948
30445    10139368
30446     6125400
30447     6373324
30448     6888144
30449     9227657
30450    12610000
30451     2394300
30452     6800000
30453     4066740
30454     6300000
30455     9014972
30456     1000000
30457     7800000
30458     6370777
30459     5778893
30460     9500000
30461     5000000
30462    10544070
30463    12000000
30464    10262010
30465     

In [57]:
df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)
print df_train.shape
print df_test.shape

# Build df_all = (df_train+df_test).join(df_macro)
num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
print df_all.shape
df_all = pd.merge(df_all, df_macro, on='timestamp', how='left')
print(df_all.shape)
df_all.head()  




(30471, 290)
(7662, 290)
(38133, 290)
(38133, 303)


,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,average_provision_of_build_contract,micex_rgbi_tr,micex_cbi_tr,deposits_rate,mortgage_value,mortgage_rate,income_per_cap,rent_price_4+room_bus,museum_visitis_per_100_cap,apartment_build
0,2011-08-20,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.76,131.16,204.78,4.1,323275,11.84,42688.6,136.11,1240.0,23587.0
1,2011-08-23,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.76,131.45,204.92,4.1,323275,11.84,42688.6,136.11,1240.0,23587.0
2,2011-08-27,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.76,131.08,204.84,4.1,323275,11.84,42688.6,136.11,1240.0,23587.0
3,2011-09-01,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.76,131.45,205.42,4.0,386388,11.92,40311.3,155.22,1240.0,23587.0
4,2011-09-05,77.0,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,5.76,131.19,198.24,4.0,386388,11.92,40311.3,155.22,1240.0,23587.0


In [58]:
df_all.head()
print df_all.shape
print df_all.dtypes




(38133, 303)
timestamp                                datetime64[ns]
full_sq                                         float64
life_sq                                         float64
floor                                           float64
max_floor                                       float64
material                                        float64
build_year                                      float64
num_room                                        float64
kitch_sq                                        float64
state                                           float64
product_type                                     object
sub_area                                         object
area_m                                          float64
raion_popul                                       int64
green_zone_part                                 float64
indust_part                                     float64
children_preschool                                int64
preschool_quota                    

In [59]:
for c in df_all.columns:
    if df_all[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_all[c].values))
        #print c+'c:  ' 
        #print lbl.classes_
        df_all[c] = lbl.transform(list(df_all[c].values))
        # x_train.drop(c,axis=1,inplace=True)
        
        

In [60]:
df_numeric = df_all.select_dtypes(exclude=['object'])
df_numeric.shape



(38133, 303)

In [61]:
df_obj = df_all.select_dtypes(include=['object']).copy()
for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]
df_obj.shape

(38133, 0)

In [62]:
df_values = pd.concat([df_numeric, df_obj], axis=1)


In [63]:
X_all = df_values.values

print(X_all.shape)

(38133, 303)


In [64]:
num_val = int(num_train * 0.2)
print num_val
print num_train
print ylog_train_all.shape

X_train_all = X_all[:num_train] 
X_train = X_all[:num_train - num_val] #训练数据
X_val = X_all[num_train - num_val:num_train] #验证数据
ylog_train = ylog_train_all[:-num_val] #训练数据标签
ylog_val = ylog_train_all[-num_val:]   #验证数据标签
X_test = X_all[num_train:]
df_columns = df_values.columns
df_columns

6094
30471
(30471,)


Index([u'timestamp', u'full_sq', u'life_sq', u'floor', u'max_floor',
       u'material', u'build_year', u'num_room', u'kitch_sq', u'state',
       ...
       u'average_provision_of_build_contract', u'micex_rgbi_tr',
       u'micex_cbi_tr', u'deposits_rate', u'mortgage_value', u'mortgage_rate',
       u'income_per_cap', u'rent_price_4+room_bus',
       u'museum_visitis_per_100_cap', u'apartment_build'],
      dtype='object', length=303)

In [65]:
print('X_train_all shape is', X_train_all.shape)
print('X_train shape is', X_train.shape)
print('y_train shape is', ylog_train.shape)
print('X_val shape is', X_val.shape)
print('y_val shape is', ylog_val.shape)
print('X_test shape is', X_test.shape)

('X_train_all shape is', (30471, 303))
('X_train shape is', (24377, 303))
('y_train shape is', (24377,))
('X_val shape is', (6094, 303))
('y_val shape is', (6094,))
('X_test shape is', (7662, 303))


In [66]:
dtrain_all = xgb.DMatrix(X_train_all, ylog_train_all, feature_names=df_columns) #所有数据
dtrain = xgb.DMatrix(X_train, ylog_train, feature_names=df_columns)  #所有数据0.8训练
dval = xgb.DMatrix(X_val, ylog_val, feature_names=df_columns)    #所有数据0.2验证
dtest = xgb.DMatrix(X_test, feature_names=df_columns)

TypeError: float() argument must be a string or a number

In [ ]:
xgb_params = {
    'eta': 0.1,  #学习率
    'max_depth': 5,#构建树的深度，越大越容易拟合
    'subsample': 1.0, #随机采样训练样本
    'colsample_bytree': 0.7,#生成树的列采样
    'objective': 'reg:linear',#线性回归
    'eval_metric': 'rmse',
    'silent': 0  #设置成1则没有运行信息输出，最好是设置为0
}


In [ ]:
partial_model = xgb.train(xgb_params, dtrain, num_boost_round=1000, evals=[(dval, 'val')],
                          early_stopping_rounds=20, verbose_eval=20)
fig, ax = plt.subplots(1, 1, figsize=(8, 16))
xgb.plot_importance(partial_model, max_num_features=50, height=0.5, ax=ax)



In [ ]:
num_boost_round = partial_model.best_iteration

model = xgb.train(dict(xgb_params, silent=0), dtrain_all, num_boost_round=num_boost_round)
fig, ax = plt.subplots(1, 1, figsize=(8, 16))
xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)




In [ ]:

ylog_pred = model.predict(dtest)
y_pred = np.exp(ylog_pred) - 1
df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})


In [ ]:
ddtrain = xgb.DMatrix(X_train_all, ylog_train_all)
ddtest = xgb.DMatrix(X_test)

In [ ]:
cv_output = xgb.cv(xgb_params, ddtrain, num_boost_round=1000, early_stopping_rounds=20,
                   verbose_eval=50, show_stdv=False,nfold=10)
cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()


In [ ]:
num_boost_rounds = len(cv_output)
model = xgb.train(dict(xgb_params, silent=0), ddtrain, num_boost_round=num_boost_rounds)

fig, ax = plt.subplots(1, 1, figsize=(8, 13))
xgb.plot_importance(model, max_num_features=50, height=0.5, ax=ax)


In [ ]:
y_predict = model.predict(ddtest)

y_pred = np.exp(y_predict) - 1

df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})

df_sub.to_csv('/Users/yangyangwang/kaggle/Sberbank-Russian-Housing-Market/subtest.csv', index=False)
